In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


# Configure an LLM
llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0.5,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    streaming=True,
    extra_body={"enable_thinking": False},
)

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


print(amultiply.name)
print(amultiply.description)
print(amultiply.args)

In [ ]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

In [ ]:
tools = [calculator]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

response = llm_with_tools.invoke(query)

response

In [ ]:
response.tool_calls

In [ ]:
messages = []

for tool_call in response.tool_calls:
    selected_tool = {"multiply": calculator, "amultiply": calculator}[
        tool_call["name"].lower()
    ]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

In [ ]:
llm_with_tools.invoke(messages)

In [ ]:
llm_with_tools.invoke([response] + messages)